In [110]:
from bs4 import BeautifulSoup #网页解析
import re #正则，文字匹配
import urllib.request,urllib.error #制定url，获取网页数据
import xlwt #excel操作
import sqlite3 #数据库操作
import time

In [111]:
#爬取网页

def getData(baseURL):
    count = 0;
    dataList = []
    for i in range(0,263):#n页留言，每页100条
        url = baseURL + str(i*100)
        html = askURL(url)
        #解析数据
        soup = BeautifulSoup(html,"html.parser")
        for item in soup.find_all("div",class_="reply-doc content"):
            data = []
            user_id = item.a.attrs['href']
            user_id = user_id.split(r'/')[4]
            user_name = item.find_all("a",class_="")[0].get_text()
            updata_time = item.find_all("span",class_="pubtime")[0].get_text()
            comment = item.find_all("p",class_="reply-content")[0].get_text()
            
            count+=1
            data.append(user_id)
            data.append(user_name)
            data.append(updata_time)
            data.append(comment)
            
            dataList.append(data)
        print(i)
        time.sleep(5) 
    return dataList,count

In [112]:
#获取单个网页
def askURL(url):
    #数据头，伪装浏览器
    head = {
        'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:23.0) Gecko/20100101 Firefox/23.0'
    }
    #请求对象
    req = urllib.request.Request(url=url,headers=head)
    html = ""
    try:
        response = urllib.request.urlopen(req)
        html = response.read().decode("utf-8")
    except urllib.error.URLError as e:
        if hasattr(e,"code"):
            print(e.code)
        if hasattr(e,"reasen"):
            print(e.reason)
    
    return html

In [113]:
#保存数据到数据库
def saveData2DB(datalist,dbpath):
    init_db(dbpath)
    conn = sqlite3.connect(dbpath)
    cursor = conn.cursor()
    
    for data in datalist:
        sql = 'insert into npc(user_id,user_name,updata_time) values("%s","%s","%s")'%(data[0].strip('"'),data[1].strip('"'),data[2].strip('"'))
        cursor.execute(sql)
        conn.commit()
    cursor.close()
    conn.close()

In [114]:
def init_db(dbpath):
    sql = '''
        CREATE TABLE npc
        (
            id INTEGER  PRIMARY KEY AUTOINCREMENT,
            user_id varchar,
            user_name varchar,
            updata_time varchar,
            comment varchar
        )
    '''
    conn = sqlite3.connect(dbpath) #文件存在即打开，不存在即创建
    cursor = conn.cursor()
    cursor.execute(sql)
    conn.commit()
    conn.close()

In [115]:
baseURL = "https://www.douban.com/group/topic/154877308/?start="
datalist,count = getData(baseURL)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262


In [118]:
print(count)

26269


In [117]:
dbpath = "doubangroup.db"
saveData2DB(datalist,dbpath)